In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/janatahackhealthcareanalytics/sample_submmission.csv
/kaggle/input/janatahackhealthcareanalytics/test_l0Auv8Q.csv
/kaggle/input/janatahackhealthcareanalytics/Train_2/Train/Train.csv
/kaggle/input/janatahackhealthcareanalytics/Train_2/Train/Data_Dictionary.xlsx
/kaggle/input/janatahackhealthcareanalytics/Train_2/Train/Second_Health_Camp_Attended.csv
/kaggle/input/janatahackhealthcareanalytics/Train_2/Train/First_Health_Camp_Attended.csv
/kaggle/input/janatahackhealthcareanalytics/Train_2/Train/Patient_Profile.csv
/kaggle/input/janatahackhealthcareanalytics/Train_2/Train/Third_Health_Camp_Attended.csv
/kaggle/input/janatahackhealthcareanalytics/Train_2/Train/Health_Camp_Detail.csv


In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_curve,roc_auc_score

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_rows' , None)
pd.set_option('display.max_columns' , None)

In [3]:
test = pd.read_csv('/kaggle/input/janatahackhealthcareanalytics/test_l0Auv8Q.csv')
train = pd.read_csv('/kaggle/input/janatahackhealthcareanalytics/Train_2/Train/Train.csv')
first_health_camp = pd.read_csv('/kaggle/input/janatahackhealthcareanalytics/Train_2/Train/First_Health_Camp_Attended.csv')
second_health_camp = pd.read_csv('/kaggle/input/janatahackhealthcareanalytics/Train_2/Train/Second_Health_Camp_Attended.csv')
patient_profile = pd.read_csv('/kaggle/input/janatahackhealthcareanalytics/Train_2/Train/Patient_Profile.csv')
third_health_camp = pd.read_csv('/kaggle/input/janatahackhealthcareanalytics/Train_2/Train/Third_Health_Camp_Attended.csv')
health_camp = pd.read_csv('/kaggle/input/janatahackhealthcareanalytics/Train_2/Train/Health_Camp_Detail.csv')
sub=pd.read_csv('/kaggle/input/janatahackhealthcareanalytics/sample_submmission.csv')

In [4]:
test.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5
0,505701,6548,21-May-06,1,0,0,0,2
1,500633,6584,02-Jun-06,0,0,0,0,0
2,506945,6582,10-Aug-06,0,0,0,0,0
3,497447,6551,27-Aug-06,0,0,0,0,0
4,496446,6533,19-Sep-06,0,0,0,0,0


In [5]:
train.dropna(inplace = True)

In [6]:
first_health_camp.head()

,Patient_ID,Health_Camp_ID,Donation,Health_Score,Unnamed: 4
0,506181,6560,40,0.439024,NaN
1,494977,6560,20,0.097561,NaN
2,518680,6560,10,0.048780,NaN
3,509916,6560,30,0.634146,NaN
4,488006,6560,20,0.024390,NaN


In [7]:
first_health_camp.drop(columns = ['Unnamed: 4'] , inplace = True)

In [8]:
import datetime
from datetime import datetime

In [9]:
def convert_datetime(x):
    
    return datetime.strptime(x ,  '%d-%b-%y')

In [10]:
train['Registration_Date'] = train['Registration_Date'].apply(convert_datetime)
test['Registration_Date'] = test['Registration_Date'].apply(convert_datetime)

In [11]:
health_camp['Camp_Start_Date'] = health_camp['Camp_Start_Date'].apply(convert_datetime)

In [12]:
patient_profile['First_Interaction'] = patient_profile['First_Interaction'].apply(convert_datetime)

In [13]:
health_camp['Camp_End_Date'] = health_camp['Camp_End_Date'].apply(convert_datetime)

In [14]:
def change_to_int(r):

    if r == 'None':
        return None
    else:
        return float(r)

In [15]:
def Age_fill(x):
    if x=='None':
        return 44.0
    else:
        return int(x)
    
patient_profile['Age'] = patient_profile['Age'].apply(Age_fill)

In [16]:
patient_profile['Education_Score'] = patient_profile['Education_Score'].apply(change_to_int)

In [17]:
patient_profile['Education_Score'] = pd.to_numeric(patient_profile['Education_Score'] , errors = 'coerce').fillna(0).astype(np.float64)

In [18]:
patient_profile['Income'] = patient_profile['Income'].apply(change_to_int)

In [19]:
patient_profile['Employer_Category'].fillna('Unemployed' , inplace = True)

In [20]:
patient_profile['Income'] = patient_profile['Income'].apply(lambda x: 0 if pd.isnull(x) else x)

In [21]:
patient_profile.head()

,Patient_ID,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category
0,516956,0,0,0,0,1.0,90.0,39.0,2003-06-18,NaN,Software Industry
1,507733,0,0,0,0,1.0,0.0,40.0,2003-07-20,H,Software Industry
2,508307,0,0,0,0,3.0,87.0,46.0,2002-11-02,D,BFSI
3,512612,0,0,0,0,1.0,75.0,47.0,2002-11-02,D,Education
4,521075,0,0,0,0,3.0,0.0,80.0,2002-11-24,H,Others


In [22]:
patient_profile['Income'].value_counts()

0.0    35304
1.0     1013
2.0      685
3.0      362
4.0      156
5.0       70
6.0       43
Name: Income, dtype: int64

In [23]:
patient_profile.groupby(['Income'])['City_Type'].value_counts()

Income  City_Type
0.0     H            2257
        B            2171
        A            1652
        D            1423
        G            1091
        E             986
        C             962
        I             826
        F             684
1.0     B             167
        H             144
        C             124
        D             107
        G             105
        E              93
        I              83
        F              60
        A              47
2.0     B             183
        D             101
        G              80
        H              76
        E              69
        I              67
        C              59
        F              26
        A               8
3.0     B              79
        D              50
        E              46
        C              39
        G              38
        I              32
        F              28
        H              23
        A               8
4.0     B              36
        I           

In [24]:
def fill_city(x):
    City = x[0]
    Income = x[1]
    if pd.isnull(City):
        if Income ==0:
            return 'H'
        else:
            return 'B'
        
    else:
        return City

In [25]:
patient_profile['City_Type'] = patient_profile[['City_Type' , 'Income']].apply(fill_city,axis=1)

In [26]:
df=pd.merge(train,health_camp,how='left',on=['Health_Camp_ID'])
df_test=pd.merge(test,health_camp,how='left',on=['Health_Camp_ID'])

In [27]:
df_1=pd.merge(df,patient_profile,how='left',on=['Patient_ID'])
df_test_final=pd.merge(df_test,patient_profile,how='left',on=['Patient_ID'])

In [28]:
df.isnull().sum()

Patient_ID           0
Health_Camp_ID       0
Registration_Date    0
Var1                 0
Var2                 0
Var3                 0
Var4                 0
Var5                 0
Camp_Start_Date      0
Camp_End_Date        0
Category1            0
Category2            0
Category3            0
dtype: int64

In [29]:
df_1.isnull().sum()

Patient_ID           0
Health_Camp_ID       0
Registration_Date    0
Var1                 0
Var2                 0
Var3                 0
Var4                 0
Var5                 0
Camp_Start_Date      0
Camp_End_Date        0
Category1            0
Category2            0
Category3            0
Online_Follower      0
LinkedIn_Shared      0
Twitter_Shared       0
Facebook_Shared      0
Income               0
Education_Score      0
Age                  0
First_Interaction    0
City_Type            0
Employer_Category    0
dtype: int64

In [30]:
df_test.isnull().sum()

Patient_ID           0
Health_Camp_ID       0
Registration_Date    0
Var1                 0
Var2                 0
Var3                 0
Var4                 0
Var5                 0
Camp_Start_Date      0
Camp_End_Date        0
Category1            0
Category2            0
Category3            0
dtype: int64

In [31]:
df_test_final.isnull().sum()

Patient_ID           0
Health_Camp_ID       0
Registration_Date    0
Var1                 0
Var2                 0
Var3                 0
Var4                 0
Var5                 0
Camp_Start_Date      0
Camp_End_Date        0
Category1            0
Category2            0
Category3            0
Online_Follower      0
LinkedIn_Shared      0
Twitter_Shared       0
Facebook_Shared      0
Income               0
Education_Score      0
Age                  0
First_Interaction    0
City_Type            0
Employer_Category    0
dtype: int64

In [32]:
df_2=pd.merge(df_1,first_health_camp,how='left',on=['Patient_ID','Health_Camp_ID'])

In [33]:
df_3=pd.merge(df_2,second_health_camp,how='left',on=['Patient_ID', 'Health_Camp_ID'])

In [34]:
df_final=pd.merge(df_3,third_health_camp,how='left',on=['Patient_ID', 'Health_Camp_ID'])

In [35]:
df_final.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category,Donation,Health_Score,Health Score,Number_of_stall_visited,Last_Stall_Visited_Number
0,489652,6578,2005-09-10,4,0,0,0,2,2005-08-16,2005-10-14,Third,G,2,0,0,0,0,0.0,0.0,44.0,2004-12-06,H,Unemployed,NaN,NaN,NaN,2.0,1.0
1,507246,6578,2005-08-18,45,5,0,0,7,2005-08-16,2005-10-14,Third,G,2,0,0,0,0,1.0,75.0,40.0,2004-09-08,C,Others,NaN,NaN,NaN,NaN,NaN
2,523729,6534,2006-04-29,0,0,0,0,0,2005-10-17,2007-11-07,Second,A,2,0,0,0,0,0.0,0.0,44.0,2004-06-22,H,Unemployed,NaN,NaN,0.402054,NaN,NaN
3,524931,6535,2004-02-07,0,0,0,0,0,2004-02-01,2004-02-18,First,E,2,0,0,0,0,0.0,0.0,44.0,2004-02-07,I,Unemployed,NaN,NaN,NaN,NaN,NaN
4,521364,6529,2006-02-28,15,1,0,0,7,2006-03-30,2006-04-03,Second,A,2,0,0,0,1,1.0,70.0,40.0,2003-07-04,I,Technology,NaN,NaN,0.845597,NaN,NaN


In [36]:
df_final[df_final['Health Score'] != None]['Health Score'].value_counts()

0.402054    1018
0.373206     545
0.455371     135
0.505134     126
0.065742     109
0.572376     107
0.691194     105
0.449940     100
0.507841      96
0.747286      93
0.803378      93
0.627865      92
0.389626      85
0.338359      80
0.102063      75
0.423103      73
0.472317      72
0.586493      71
0.532780      70
0.845597      70
0.796998      69
0.884801      65
0.558452      65
0.113106      63
0.844787      63
0.103136      62
0.290109      62
0.610190      60
0.867694      58
0.819905      58
0.908768      56
0.917973      55
0.252714      52
0.754739      51
0.886651      48
0.615827      47
0.628357      46
0.944511      44
0.692998      44
0.548872      44
0.722041      43
0.552962      42
0.734597      42
0.687599      40
0.674820      40
0.518797      40
0.718009      39
0.058993      39
0.575530      39
0.702607      38
0.769747      38
0.503597      38
0.816876      37
0.642970      37
0.125452      37
0.221351      37
0.923381      37
0.673181      36
0.776978      

In [37]:
df_final[df_final['Health Score'] != None]['Health Score'].mean()

0.5556291033562536

In [38]:
df_final[df_final['Health Score'] != None]['Health Score'].max()

1.0

In [39]:
df_final[df_final['Health_Score'] != None]['Health_Score'].value_counts()

0.360339    111
0.464856    102
0.559855    101
0.753325     91
0.752137     58
0.887545     56
0.436464     56
0.642173     53
0.696429     51
0.779553     51
0.629834     37
0.880192     34
1.000000     34
0.897790     28
0.776243     27
0.204353     25
0.039936     25
0.226837     23
0.286667     21
0.942492     21
0.833333     20
0.250000     19
0.160822     19
0.101083     18
0.775510     18
0.282748     18
0.500000     18
0.958888     16
0.018138     15
0.563636     14
0.259669     14
0.078597     14
0.377593     14
0.367052     13
0.161538     13
0.330049     13
0.033149     12
0.888889     12
0.219653     11
0.372549     11
0.196429     11
0.198895     11
0.158333      9
0.182109      9
0.114108      9
0.745455      9
0.118211      9
0.285714      9
0.270758      9
0.777778      8
0.750000      8
0.357143      8
0.853933      8
0.444444      8
0.372093      8
0.666667      8
0.227848      8
0.857143      7
0.142857      7
0.428571      7
0.162393      7
0.985490      7
0.611111

In [41]:
def find_outcomes(x):
    first = x[0]
    second = x[1]
    third = x[2]
    if pd.isnull(first) == False:
        return 1
    elif pd.isnull(second) == False:
        return 1
    elif pd.isnull(third) == False:
        if third > 0:
            return 1
        else:
            return 0
    else:
        return 0
        

In [42]:
df_final['Outcomes'] = df_final[['Health_Score', 'Health Score', 'Number_of_stall_visited']].apply(find_outcomes , axis =1)

In [43]:
df_final.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category,Donation,Health_Score,Health Score,Number_of_stall_visited,Last_Stall_Visited_Number,Outcomes
0,489652,6578,2005-09-10,4,0,0,0,2,2005-08-16,2005-10-14,Third,G,2,0,0,0,0,0.0,0.0,44.0,2004-12-06,H,Unemployed,NaN,NaN,NaN,2.0,1.0,1
1,507246,6578,2005-08-18,45,5,0,0,7,2005-08-16,2005-10-14,Third,G,2,0,0,0,0,1.0,75.0,40.0,2004-09-08,C,Others,NaN,NaN,NaN,NaN,NaN,0
2,523729,6534,2006-04-29,0,0,0,0,0,2005-10-17,2007-11-07,Second,A,2,0,0,0,0,0.0,0.0,44.0,2004-06-22,H,Unemployed,NaN,NaN,0.402054,NaN,NaN,1
3,524931,6535,2004-02-07,0,0,0,0,0,2004-02-01,2004-02-18,First,E,2,0,0,0,0,0.0,0.0,44.0,2004-02-07,I,Unemployed,NaN,NaN,NaN,NaN,NaN,0
4,521364,6529,2006-02-28,15,1,0,0,7,2006-03-30,2006-04-03,Second,A,2,0,0,0,1,1.0,70.0,40.0,2003-07-04,I,Technology,NaN,NaN,0.845597,NaN,NaN,1


In [44]:
drop_cols=['Camp_Start_Date','Camp_End_Date','Patient_ID','Health_Camp_ID','Donation','Health_Score','Health Score','Number_of_stall_visited','Last_Stall_Visited_Number'
          ,'Registration_Date' , 'First_Interaction' ]
drop_cols_test=['Patient_ID','Health_Camp_ID' , 'Camp_End_Date' , 'Camp_Start_Date' , 'Registration_Date' , 'First_Interaction' ]

In [45]:
train_final=df_final.drop(columns=drop_cols)
test_final=df_test_final.drop(columns=drop_cols_test)

In [46]:
train_final.shape

(74944, 18)

In [47]:
train_final.head()

,Var1,Var2,Var3,Var4,Var5,Category1,Category2,Category3,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,City_Type,Employer_Category,Outcomes
0,4,0,0,0,2,Third,G,2,0,0,0,0,0.0,0.0,44.0,H,Unemployed,1
1,45,5,0,0,7,Third,G,2,0,0,0,0,1.0,75.0,40.0,C,Others,0
2,0,0,0,0,0,Second,A,2,0,0,0,0,0.0,0.0,44.0,H,Unemployed,1
3,0,0,0,0,0,First,E,2,0,0,0,0,0.0,0.0,44.0,I,Unemployed,0
4,15,1,0,0,7,Second,A,2,0,0,0,1,1.0,70.0,40.0,I,Technology,1


In [48]:
test_final.shape

(35249, 17)

In [49]:
label_cols = ['Category1','Category2']

In [50]:
encoder = LabelEncoder()

In [51]:
for item in label_cols:
    train_final[item] = encoder.fit_transform(train_final[item])
    test_final[item] = encoder.transform(test_final[item])

In [52]:
train_final.head()

,Var1,Var2,Var3,Var4,Var5,Category1,Category2,Category3,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,City_Type,Employer_Category,Outcomes
0,4,0,0,0,2,2,6,2,0,0,0,0,0.0,0.0,44.0,H,Unemployed,1
1,45,5,0,0,7,2,6,2,0,0,0,0,1.0,75.0,40.0,C,Others,0
2,0,0,0,0,0,1,0,2,0,0,0,0,0.0,0.0,44.0,H,Unemployed,1
3,0,0,0,0,0,0,4,2,0,0,0,0,0.0,0.0,44.0,I,Unemployed,0
4,15,1,0,0,7,1,0,2,0,0,0,1,1.0,70.0,40.0,I,Technology,1


In [53]:
dummie_cols=['City_Type','Employer_Category']

In [54]:
train_final=pd.get_dummies(train_final,columns=dummie_cols,drop_first=True)
test_final=pd.get_dummies(test_final,columns=dummie_cols,drop_first=True)

In [55]:
train_final.head()

,Var1,Var2,Var3,Var4,Var5,Category1,Category2,Category3,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,Outcomes,City_Type_B,City_Type_C,City_Type_D,City_Type_E,City_Type_F,City_Type_G,City_Type_H,City_Type_I,Employer_Category_Broadcasting,Employer_Category_Consulting,Employer_Category_Education,Employer_Category_Food,Employer_Category_Health,Employer_Category_Manufacturing,Employer_Category_Others,Employer_Category_Real Estate,Employer_Category_Retail,Employer_Category_Software Industry,Employer_Category_Technology,Employer_Category_Telecom,Employer_Category_Transport,Employer_Category_Unemployed
0,4,0,0,0,2,2,6,2,0,0,0,0,0.0,0.0,44.0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,45,5,0,0,7,2,6,2,0,0,0,0,1.0,75.0,40.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,2,0,0,0,0,0.0,0.0,44.0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,4,2,0,0,0,0,0.0,0.0,44.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,15,1,0,0,7,1,0,2,0,0,0,1,1.0,70.0,40.0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [56]:
train_final.isnull().sum()

Var1                                   0
Var2                                   0
Var3                                   0
Var4                                   0
Var5                                   0
Category1                              0
Category2                              0
Category3                              0
Online_Follower                        0
LinkedIn_Shared                        0
Twitter_Shared                         0
Facebook_Shared                        0
Income                                 0
Education_Score                        0
Age                                    0
Outcomes                               0
City_Type_B                            0
City_Type_C                            0
City_Type_D                            0
City_Type_E                            0
City_Type_F                            0
City_Type_G                            0
City_Type_H                            0
City_Type_I                            0
Employer_Categor

In [57]:
X=train_final.drop('Outcomes',axis=1)
y=train_final['Outcomes']

In [58]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=101)

In [59]:
scaler=MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)
test_final=scaler.transform(test_final)

In [60]:
from sklearn.metrics import roc_auc_score , roc_curve

In [61]:
import xgboost as xgb

In [62]:
DMat = xgb.DMatrix(X_train , label = y_train)

In [63]:
model_xgb = xgb.XGBClassifier()

In [ ]:
model_xgb.fit(X_train , y_train)

In [ ]:
pred_xgb = model_xgb.predict_proba(X_test)[:,-1]

In [ ]:
roc_xgb = roc_auc_score(y_test , pred_xgb)

In [ ]:
roc_xgb

In [ ]:
train_final.head()

In [ ]:
sub.head()

In [ ]:
pred_test_xgb = model_xgb.predict_proba(test_final)[:,-1]

In [ ]:
sub['Outcome'] = pred_test_xgb

In [ ]:
sub.to_csv('XGB3_HealthCare.csv',index=False)